# LLamaIndex + GitHub Models for RAG

This notebook demonstrates how to perform Retrieval-Augmented Generation (RAG) with LLamaIndex.

## Introduction

Retrieval-Augmented Generation (RAG) is a technique in natural language processing that combines the strengths of retrieval-based and generation-based methods to enhance the quality and accuracy of generated text. It integrates a retriever module, which searches a large corpus of documents for relevant information, with a generator module to produce coherent and contextually appropriate responses. This hybrid approach allows RAG to leverage vast amounts of external knowledge stored in documents, making it particularly effective for tasks requiring detailed information and context beyond the model's pre-existing knowledge.

RAG operates by first using the retriever to identify the most relevant pieces of information from a database or collection of texts. These retrieved passages are then fed into the generator, which synthesizes the information to produce a final response. This process enables the model to provide more accurate and informative answers, as it dynamically incorporates up-to-date and specific details from the retrieval stage. The combination of retrieval and generation ensures that RAG models are both knowledgeable and flexible, making them valuable for applications such as question answering, summarization, and dialogue systems.

In this sample, we will create an index from a set of markdown documents that contain product descriptions. Using a retriever, we will search the index with a user question to find the most relevant documents. Then we will use llama-index's query for a full Retrieval-Augmented Generation (RAG) implementation.

## 1. Install dependencies

In [2]:
%pip install llama-index
%pip install openai
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 2. Setup classes to a chat model and an embedding model

To run RAG, you need 2 models: a chat model, and an embedding model. The GitHub Model service offers different options.

For instance you could use an Azure OpenAI chat model (`gpt-4o-mini`) and embedding model (`text-embedding-3-small`), or a Cohere chat model (`Cohere-command-r-plus`) and embedding model (`Cohere-embed-v3-multilingual`).

We'll proceed using some of the Azure OpenAI models below. You can find [how to leverage Cohere models in the LlamaIndex documentation](https://docs.llamaindex.ai/en/stable/examples/llm/cohere/).

### Example using Azure OpenAI models

In [3]:
import os
import dotenv

dotenv.load_dotenv()

if not os.getenv("GITHUB_TOKEN"):
    raise ValueError("GITHUB_TOKEN is not set")

os.environ["OPENAI_API_KEY"] = os.getenv("GITHUB_TOKEN")
os.environ["OPENAI_BASE_URL"] = "https://models.inference.ai.azure.com/"

Below, we are setting up the embedding model and the llm model to be used. 

In [4]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings
import logging
import sys, os

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

llm = OpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    api_base=os.getenv("OPENAI_BASE_URL"),
)

embed_model = OpenAIEmbedding(
    model="text-embedding-3-small",
    api_key=os.getenv("OPENAI_API_KEY"),
    api_base=os.getenv("OPENAI_BASE_URL"),
)

Settings.llm = llm
Settings.embed_model = embed_model

## 3. Create an index and retriever

In the data folder, we have some product information files in markdown format. Here is a sample of the data:

```markdown
# Information about product item_number: 1
TrailMaster X4 Tent, price $250,


## Brand
OutdoorLiving

Main Category: CAMPING & HIKING
Sub Category: TENTS & SHELTERS
Product Type: BACKPACKING TENTS

## Features
- Polyester material for durability
- Spacious interior to accommodate multiple people
- Easy setup with included instructions
- Water-resistant construction to withstand light rain
- Mesh panels for ventilation and insect protection
- Rainfly included for added weather protection
- Multiple doors for convenient entry and exit
- Interior pockets for organizing small items
- Reflective guy lines for improved visibility at night
- Freestanding design for easy setup and relocation
- Carry bag included for convenient storage and transportation
```
Here is the link to the full file: [data/product_info_1.md](data/product_info_1.md). As you can see, the files are rather long and contain different sections like Brand, Features, User Guide, Warranty Information, Reviews, etc. All these can be useful when answering user questions.

To be able to find the right information, we will create a vector index that stores the embeddings of the documents. Note that we are reducing the batch size of the indexer to prevent rate limiting. The GitHub Model Service is rate limited to 64K tokens per request for embedding models.  


In [5]:
#Note: we have to reduce the batch size to stay within the token limits of the free service
documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents, insert_batch_size=150)

INFO:httpx:HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"


Now that we have an index, we can use the retriever to find the most relevant documents for a user question.

In [6]:
retriever = index.as_retriever()
#fragments = retriever.retrieve("What is the temperature rating of the cozynights sleeping bag?")
fragments = retriever.retrieve("What is the temperature rating of the Folding Table sleeping bag?")

for fragment in fragments:
    print(fragment)

INFO:httpx:HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"
Node ID: 9a50a4cf-f48d-43c8-8ce3-263e434e8ae1
Text: 3. Temperature Rating and Comfort: The MountainDream Sleeping
Bag is rated for temperatures between 15°F to 30°F. However, personal
comfort preferences may vary. It is recommended to use additional
layers or adjust ventilation using the zipper and hood to achieve the
desired temperature.
Score:  0.623

Node ID: 1a907901-927d-4054-a1b0-6e13ed1db975
Text: FAQ 31) What is the temperature rating of the CozyNights
Sleeping Bag?    The CozyNights Sleeping Bag is rated for 3-season use
and has a temperature rating of 20�F to 60�F (-6�C to 15�C).  32) Can
the CozyNights Sleeping Bag be zipped together with another sleeping
bag to create a double sleeping bag?    Yes, two CozyNights Sleeping
Bags can be...
Score:  0.617



## 4. Use chat model to generate an answer

Now that we have the documents that match the user question, we can ask our chat model to generate an answer based on the retrieved documents:

In [7]:
from llama_index.core.llms import ChatMessage

context = "\n------\n".join([ fragment.text for fragment in fragments ])

messages = [
    ChatMessage(role="system", content="You are a helpful assistant that answers some questions with the help of some context data.\n\nHere is the context data:\n\n" + context),
    ChatMessage(role="user", content="What is the temperature rating of the Folding Table sleeping bag?")
    #ChatMessage(role="user", content="What is the temperature rating of the cozynights sleeping bag?")
]

response = llm.chat(messages)
print()
print(response)

INFO:httpx:HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"

assistant: The context data does not provide information about the temperature rating of the Folding Table sleeping bag. You may need to check the manufacturer's specifications or product details for that information.


LLamaIndex provides a simple API to query the retriever and the generator in one go. The query function takes the question as input and returns the answer generated by the generator.

In [8]:
query_engine = index.as_query_engine()
response = query_engine.query("What is the temperature rating of the cozynights sleeping bag?")
print()
print(response)

INFO:httpx:HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"

The CozyNights Sleeping Bag is rated for 3-season use with a temperature range of 20°F to 60°F (-6°C to 15°C).


In [9]:
response = query_engine.query("What is a good 2 person tent?")
print(response)

INFO:httpx:HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"
A good option for a 2-person tent is the SkyView 2-Person Tent. It is praised for its spaciousness, ease of setup, and durability. Users have highlighted its excellent ventilation, waterproof design, and thoughtful storage features, making it suitable for various camping conditions.


In [10]:
response = query_engine.query("Does the SkyView 2-Person Tent have a rain fly?")
print(response)

INFO:httpx:HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"
Yes, the SkyView 2-Person Tent includes a rainfly as one of its components.
